In [1]:
import pandas as pd # Manejo de bases de datos
import geopandas as gpd # Manejo de bases de datos geográficas
import numpy as np # Funciones numéricas
import matplotlib.pyplot as plt # Gráficas
import seaborn as sns # Gráficas
import datetime as dt
import folium
import unicodedata
import datetime as dt

from geopandas.tools import sjoin
from unicodedata import normalize

## Read UTAM areas

In [2]:
# Load file
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/UTAM/UTAM.shp'
utam_df = gpd.read_file(path)

utam_df.to_crs(epsg=4326, inplace=True)
utam_df.head(2)

,MUNCodigo,MUNNombre,LOCNombre,USOSNum,USOPreNum,USOPreCor,ESTRATO1,ESTRATO2,ESTRATO3,ESTRATO4,ESTRATO5,ESTRATO6,ESTRATOPre,HOGARES,UTAM,UTAMNombre,UTAMArea,geometry
0,11001.0,BOGOTA,TEUSAQUILLO,12345.0,1.0,RESIDENCIAL,NO,NO,SI,SI,NO,NO,4.0,14357.0,UTAM100,GALERIAS,2.372501e+06,"POLYGON ((-74.06463 4.64965, -74.06619 4.64146..."
1,11001.0,BOGOTA,KENNEDY,12345.0,2.0,COMERCIO Y SERVICIOS,SI,SI,NO,NO,NO,NO,2.0,3978.0,UTAM83,LAS MARGARITAS,1.470364e+06,"POLYGON ((-74.17077 4.62848, -74.17141 4.62811..."


In [3]:
# Drop non urbanized areas
utam_df = utam_df[ utam_df['LOCNombre'] != 'SUMAPAZ' ]
utam_df = utam_df[ utam_df['UTAMNombre'] != 'N/A' ] # non urbanized areas, areas not belonging to Bogotá
utam_df = utam_df[ utam_df['UTAMNombre'] != 'UPR CERROS ORIENTALES' ] #non urbanized areas
utam_df = utam_df[ utam_df['UTAMNombre'] != 'UPR ZONA NORTE' ] #non urbanized areas

utam_df.drop(columns={'MUNNombre', 'MUNCodigo', 'USOSNum', 'USOPreNum', 'ESTRATO1', 'ESTRATO2',
                     'ESTRATO3', 'ESTRATO4', 'ESTRATO5', 'ESTRATO6', 'USOPreCor'},inplace=True)

In [4]:
# take out the 'UTAM' text in UTAM column and cast to numeric value
utam_df['UTAM'] = utam_df.apply(lambda row: row['UTAM'][4:], axis=1)
utam_df['UTAM'] = pd.to_numeric(utam_df['UTAM'])
# ascending UTAM sort
utam_df.sort_values(by='UTAM', ascending=True, inplace=True)
utam_df.reset_index(drop=True, inplace=True)
# lower case text standarization
utam_df['LOCNombre'] = utam_df.apply(lambda row: row['LOCNombre'].lower(), axis=1)

utam_df.iloc[0:5, :]

,LOCNombre,ESTRATOPre,HOGARES,UTAM,UTAMNombre,UTAMArea,geometry
0,usaquen,6.0,913.0,1,PASEO DE LOS LIBERTADORES,6.301165e+06,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311..."
1,suba,6.0,891.0,2,LA ACADEMIA,6.711816e+06,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702..."
2,suba,6.0,452.0,3,GUAYMARAL,4.530358e+06,"POLYGON ((-74.03501 4.82553, -74.03501 4.82552..."
3,usaquen,3.0,28882.0,9,VERBENAL,3.553194e+06,"MULTIPOLYGON (((-74.02714 4.76985, -74.02709 4..."
4,usaquen,4.0,6461.0,10,LA URIBE,3.448096e+06,"POLYGON ((-74.02110 4.75705, -74.02110 4.75704..."


In [5]:
len(utam_df['LOCNombre']) # Number of UTAMS

112

### Bogotá spatial characteristics 

In [6]:
# Load file
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/1_spatial/Outputs/ManzanasGDF.gzip'
spatial_Bog = pd.read_pickle(path, compression='gzip')
# drop non needed cols
spatial_Bog.drop(columns={'OBJECTID', 'GLOBALID', 'Shape_Leng', 'Shape_Area', 'SECCODIGO'}, inplace=True)

spatial_Bog.head(2)

,MANCODIGO,geometry,Hosp_Point,N_Hosp,IPS_Point,N_IPS,Col_Point,N_Col,PlazMer_Point,N_PlazMer,ITur_Point,N_ITur,SITP_Point,N_SITP,Ecomer_Point,N_Ecomer
0,001101001,"POLYGON ((-74.08180 4.58640, -74.08180 4.58640...",[],0,[],0,[],0,[],0,[],0,0,0,"[POINT (-74.0818740288 4.58465360735), POINT (...",11
1,001101002,"POLYGON ((-74.08168 4.58489, -74.08168 4.58490...",[],0,[],0,[],0,[],0,[],0,[POINT (-74.08057397553043 4.585286425284456)],1,"[POINT (-74.08039633520001 4.583938352930001),...",24


### Create grouped dataframe with utam zones and spatial characteristics

In [7]:
# Perform spatial join
join_spatial_utam = sjoin(utam_df, spatial_Bog, how="left", op="contains")
# drop point cols we dont need them now

join_spatial_utam.drop(columns={'Hosp_Point', 'IPS_Point', 'Col_Point', 'PlazMer_Point',
                               'ITur_Point', 'SITP_Point', 'Ecomer_Point', 'index_right',
                               'MANCODIGO'}, inplace=True)

join_spatial_utam.head(2)

,LOCNombre,ESTRATOPre,HOGARES,UTAM,UTAMNombre,UTAMArea,geometry,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer
0,usaquen,6.0,913.0,1,PASEO DE LOS LIBERTADORES,6.301165e+06,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,0,0,0,26
0,usaquen,6.0,913.0,1,PASEO DE LOS LIBERTADORES,6.301165e+06,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,0,0,0,1


In [8]:
# Sum features that share same index (same UTAM)
spatial_utam = join_spatial_utam.groupby(join_spatial_utam.index).agg({'LOCNombre': 'first', 
                                                                       'ESTRATOPre': 'first',
                                                                       'HOGARES': 'first',
                                                                       'UTAM': 'first',
                                                                       'UTAMNombre': 'first',
                                                                       'UTAMArea': 'first',
                                                                       'geometry': 'first', 
                                                                       'N_Hosp':sum,
                                                                       'N_IPS':sum,
                                                                       'N_Col':sum,
                                                                       'N_PlazMer':sum,
                                                                       'N_ITur':sum,
                                                                       'N_ITur':sum,
                                                                       'N_SITP':sum,
                                                                       'N_Ecomer':sum})

In [9]:
# Organize cols
spatial_utam = spatial_utam[ ['LOCNombre', 'ESTRATOPre', 'HOGARES', 'UTAM', 'UTAMNombre', 'UTAMArea', 
                             'geometry', 'N_Hosp', 'N_IPS', 'N_Col', 'N_PlazMer', 'N_ITur', 'N_SITP', 
                             'N_Ecomer'] ]

spatial_utam.head(2)

,LOCNombre,ESTRATOPre,HOGARES,UTAM,UTAMNombre,UTAMArea,geometry,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer
0,usaquen,6.0,913.0,1,PASEO DE LOS LIBERTADORES,6.301165e+06,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,0,0,2,416
1,suba,6.0,891.0,2,LA ACADEMIA,6.711816e+06,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",0,1,13,0,0,12,73


### Estimate utams population density

In [10]:
# Function used to standarize lower case and accent marks
def Arreglar_tilde(Texto):
    Texto = unicodedata.normalize('NFD', Texto)
    Texto = Texto.encode('ascii', 'ignore')
    Texto = Texto.decode("utf-8")
    Texto = Texto.lower()
    return(Texto)

In [12]:
# Bogotá localities population formatting, cleaning, etc ...
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/Files/Poblacion_Bogota.xlsx'
PobBog = pd.read_excel(path, sheet_name='1.1')
PobBog = PobBog.drop([PobBog.index[0], PobBog.index[1], PobBog.index[2], PobBog.index[3], PobBog.index[4], 
                      PobBog.index[25], PobBog.index[26]])

PobBog = PobBog.filter(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 17', 'Unnamed: 18'])
PobBog = PobBog.rename(columns={'Unnamed: 0': 'id localidad', 'Unnamed: 1': 'localidad',
                                     'Unnamed: 17': 'Población 2020', 'Unnamed: 18': 'Población 2021'})

PobBog['id localidad'] = pd.to_numeric(PobBog['id localidad'])
PobBog.sort_values(by='id localidad', ascending=True)
PobBog.drop(columns={'Población 2020'}, inplace=True)
PobBog.reset_index(drop=True, inplace=True)
PobBog['localidad'] = PobBog.apply(lambda row: Arreglar_tilde(row['localidad']), axis=1)

PobBog.iloc[0:5, :]

,id localidad,localidad,Población 2021
0,1,usaquen,571268.0
1,2,chapinero,173353.0
2,3,santa fe,107784.0
3,4,san cristobal,401060.0
4,5,usme,393366.0


In [13]:
# Adding local id code

# Read spatial characteristics of Bogotá
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/1_spatial/Outputs/LocalidadGDF.gzip'
LocGDF = pd.read_pickle(path, compression='gzip')
LocGDF = LocGDF.drop(columns={'Hospitals', 'IPS', 'Colegios', 'PlazaMer', 'Turismo', 'SITP', 'Comercio'})
LocGDF.drop(19, inplace=True, axis=0)

# Extract localities name and code from file 
Loc_names = LocGDF['Name'].tolist()
Loc_code = LocGDF['Id'].tolist()

del LocGDF

In [14]:
# Assign locality ID code to each UTAM in DataFrame 
def AssignCode(LOCNombre, namelist, codelist):
    for i in range(len(namelist)):
        if LOCNombre == namelist[i]:
            return codelist[i]
        
spatial_utam['LOCid'] = spatial_utam.apply(lambda row: AssignCode(row['LOCNombre'], Loc_names, Loc_code), axis=1)

spatial_utam = spatial_utam[ ['LOCNombre', 'UTAM', 'LOCid', 'ESTRATOPre', 'HOGARES', 'UTAMNombre', 
                              'UTAMArea', 'geometry', 'N_Hosp', 'N_IPS', 'N_Col', 'N_PlazMer', 
                              'N_ITur', 'N_SITP', 'N_Ecomer'] ]

spatial_utam.head(2)

,LOCNombre,UTAM,LOCid,ESTRATOPre,HOGARES,UTAMNombre,UTAMArea,geometry,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer
0,usaquen,1,1,6.0,913.0,PASEO DE LOS LIBERTADORES,6.301165e+06,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,0,0,2,416
1,suba,2,11,6.0,891.0,LA ACADEMIA,6.711816e+06,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",0,1,13,0,0,12,73


In [15]:
# Add Locality population in order to estimate utam active cases
pop_list = PobBog['Población 2021'].tolist()

def AssignPoP(ID, code_list, pop_list):
    for i in range(len(code_list)):
        if ID == code_list[i]:
            return pop_list[i]
        
spatial_utam['PopLoc'] = spatial_utam.apply(lambda row: AssignPoP(row['LOCid'], Loc_code, pop_list), axis=1)

In [16]:
# Organize cols
spatial_utam = spatial_utam[ ['LOCNombre', 'PopLoc' ,'UTAM', 'LOCid', 'ESTRATOPre', 'HOGARES', 'UTAMNombre',
                                         'UTAMArea', 'geometry', 'N_Hosp', 'N_IPS', 'N_Col',
                                         'N_PlazMer', 'N_ITur', 'N_SITP', 'N_Ecomer'] ]

spatial_utam.head(3)

,LOCNombre,PopLoc,UTAM,LOCid,ESTRATOPre,HOGARES,UTAMNombre,UTAMArea,geometry,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer
0,usaquen,571268.0,1,1,6.0,913.0,PASEO DE LOS LIBERTADORES,6.301165e+06,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,0,0,2,416
1,suba,1252675.0,2,11,6.0,891.0,LA ACADEMIA,6.711816e+06,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",0,1,13,0,0,12,73
2,suba,1252675.0,3,11,6.0,452.0,GUAYMARAL,4.530358e+06,"POLYGON ((-74.03501 4.82553, -74.03501 4.82552...",0,0,0,0,0,0,3


In [17]:
#Sum the total of homes inside each locality
LocHomes = []
for i in range(len(Loc_names)):
    temp = spatial_utam[ spatial_utam['LOCNombre'] == Loc_names[i] ]
    temp_list = temp['HOGARES'].values.tolist()
    tot_homes = sum(temp_list)
    LocHomes.append(tot_homes)

In [18]:
# Estimate a weight vector that respond to the actual demographic info of each locality
ppl_weight = []
for i in range(len(LocHomes)):
    temp = pop_list[i]/LocHomes[i]
    ppl_weight.append(temp)

In [19]:
# Estimate population density within each locality
def PopDenEst(weight, hogares, area, ID):
    x = hogares*weight[ID-1]
    y = area/(1000**2)
    z = x/y
    return z

spatial_utam['PopDen[p/km2]'] = spatial_utam.apply(lambda row: PopDenEst(ppl_weight, row['HOGARES'], 
                                                                         row['UTAMArea'], row['LOCid']), axis=1)


In [20]:
# Orgnaize cols
spatial_utam = spatial_utam[ ['LOCNombre', 'PopLoc', 'UTAM', 'LOCid', 'ESTRATOPre', 'HOGARES',
                             'UTAMNombre', 'UTAMArea', 'PopDen[p/km2]','geometry', 'N_Hosp', 'N_IPS', 'N_Col',
                             'N_PlazMer', 'N_ITur', 'N_SITP', 'N_Ecomer'] ]

spatial_utam.head(3)

,LOCNombre,PopLoc,UTAM,LOCid,ESTRATOPre,HOGARES,UTAMNombre,UTAMArea,PopDen[p/km2],geometry,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer
0,usaquen,571268.0,1,1,6.0,913.0,PASEO DE LOS LIBERTADORES,6.301165e+06,540.118849,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,0,0,2,416
1,suba,1252675.0,2,11,6.0,891.0,LA ACADEMIA,6.711816e+06,527.173778,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",0,1,13,0,0,12,73
2,suba,1252675.0,3,11,6.0,452.0,GUAYMARAL,4.530358e+06,396.206927,"POLYGON ((-74.03501 4.82553, -74.03501 4.82552...",0,0,0,0,0,0,3


### origin-destiny / mobility survey

In [21]:
# Load file
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/UTAM/ViajesEODH2019.csv'
EODBog = pd.read_csv(path, encoding='UTF-8', delimiter=";")

EODBog.head(3)

,id_hogar,id_persona,id_viaje,fecha,lugar_origen,zat_origen,p17_Id_motivo_viaje,p17_otro_motivo,hora_inicio_viaje,p28_lugar_destino,...,p34_cual_aplicacion_durante_viaje,p35_otro_desplazamiento,p36_hora_salida,f_exp,mun_origen,mun_destino,utam_origen,utam_destino,modo_principal,modo_principal_desagregado
0,33754,2,1,43669.0,1.0,204.0,5,NaN,0.333333,2.0,...,NaN,1.0,0.416667,54.286560,11001.0,11001.0,UTAM31,UTAM86,SITP Provisional,SITP - Provisional
1,33770,2,3,43669.0,2.0,299.0,6,NaN,0.625000,1.0,...,NaN,2.0,NaN,155.235707,11001.0,11001.0,UTAM105,UTAM54,Transporte informal,Bus/buseta informal/pirata/Chana
2,3882,3,1,43605.0,1.0,551.0,1,NaN,0.250000,2.0,...,NaN,1.0,0.708333,361.516192,11001.0,11001.0,UTAM84,UTAM112,SITP Zonal,SITP - Urbano (Azul)


In [22]:
# Drop unwanted cols
EODBog.drop(columns={'id_hogar', 'id_persona', 'id_viaje', 'fecha', 'lugar_origen', 
                    'zat_origen', 'p17_Id_motivo_viaje', 'p17_otro_motivo',
                    'hora_inicio_viaje', 'p28_lugar_destino', 'zat_destino',
                    'p29_id_municipio', 'p30_camino_cuadras', 'p30_camino_minutos',
                    'p31_hora_llegada', 'p32_lunes', 'p32_martes', 'p32_miercoles',
                    'p32_jueves', 'p32_viernes', 'p32_sabado', 'p32_domingo',
                    'p32_ocasional', 'p33_aplicacion_antes_viaje', 'p33_cual_aplicacion_antes_viaje',
                    'p34_aplicacion_durante_viaje', 'p34_cual_aplicacion_durante_viaje', 
                    'p35_otro_desplazamiento', 'p36_hora_salida', 'f_exp', 'mun_origen',
                    'mun_destino', 'modo_principal_desagregado'}, inplace=True)

EODBog.head(2)

,utam_origen,utam_destino,modo_principal
0,UTAM31,UTAM86,SITP Provisional
1,UTAM105,UTAM54,Transporte informal


In [23]:
# Exclude UPR (not utam administrative divisions)
EODBog = EODBog[ EODBog['utam_origen'] != 'UPR1'  ]
EODBog = EODBog[ EODBog['utam_origen'] != 'UPR2'  ]
EODBog = EODBog[ EODBog['utam_origen'] != 'UPR2'  ]

EODBog = EODBog[ EODBog['utam_destino'] != 'UPR1'  ]
EODBog = EODBog[ EODBog['utam_destino'] != 'UPR2'  ]
EODBog = EODBog[ EODBog['utam_destino'] != 'UPR3'  ]

# Take out the 'UTAM' text and cast to numeric
EODBog['utam_origen'] = EODBog.apply(lambda row: str(row['utam_origen'])[4:], axis=1)
EODBog['utam_destino'] = EODBog.apply(lambda row: str(row['utam_destino'])[4:], axis=1)

EODBog['utam_origen'] = pd.to_numeric(EODBog['utam_origen'])
EODBog['utam_destino'] = pd.to_numeric(EODBog['utam_destino'])
# Sort
EODBog.sort_values(by='utam_origen', ascending=True, inplace=True)
EODBog.reset_index(drop=True, inplace=True)

EODBog.head(2)

,utam_origen,utam_destino,modo_principal
0,1.0,9.0,Transporte Escolar
1,1.0,114.0,Transporte Escolar


In [24]:
#Count the number of trips originated by every utam
viajes_origen = EODBog.groupby(['utam_origen'])['utam_origen'].size()
viajes_origen = pd.DataFrame(viajes_origen)
viajes_origen.rename(columns={'utam_origen': 'No viajes origen'}, inplace=True)
viajes_origen.reset_index(drop=False, inplace=True)
viajes_origen['utam_origen'] = viajes_origen.apply(lambda row: int(row['utam_origen']), axis=1)

viajes_origen.iloc[0:5, :]

,utam_origen,No viajes origen
0,1,163
1,2,287
2,3,148
3,9,1065
4,10,858


In [25]:
#Count the number of trips received by every utam
viajes_recibidos = EODBog.groupby(['utam_destino'])['utam_destino'].size()
viajes_recibidos = pd.DataFrame(viajes_recibidos)
viajes_recibidos.rename(columns={'utam_destino': 'No viajes recibidos'}, inplace=True)
viajes_recibidos.reset_index(drop=False, inplace=True)
viajes_recibidos['utam_destino'] = viajes_recibidos.apply(lambda row: int(row['utam_destino']), axis=1)

viajes_recibidos.iloc[0:5, :]

,utam_destino,No viajes recibidos
0,1,166
1,2,286
2,3,151
3,9,1061
4,10,892


In [26]:
# Add originated and destined trips to general DataFrame

def AggTripOD(UTAM, Noutam, viajes):
    trips = 0
    for i in range(len(Noutam)):
        if Noutam[i] == UTAM:
            trips = viajes[i]
            break;
    return trips
    
# Add originated trips by utam

spatial_utam['originated Trips'] = spatial_utam.apply(lambda row: AggTripOD(row['UTAM'], viajes_origen['utam_origen'].tolist(),
                                                                           viajes_origen['No viajes origen'].tolist()), axis=1)

# Add received trips by utam 

spatial_utam['received Trips'] = spatial_utam.apply(lambda row: AggTripOD(row['UTAM'], viajes_recibidos['utam_destino'].tolist(),
                                                                           viajes_recibidos['No viajes recibidos'].tolist()), axis=1)

In [27]:
# Organize cols
spatial_utam = spatial_utam[ ['LOCNombre', 'PopLoc', 'UTAM', 'LOCid', 'ESTRATOPre', 'HOGARES',
                             'UTAMNombre', 'UTAMArea', 'PopDen[p/km2]', 'geometry', 
                             'originated Trips', 'received Trips', 'N_Hosp',
                             'N_IPS', 'N_Col', 'N_PlazMer', 'N_ITur', 'N_SITP', 'N_Ecomer',] ]

spatial_utam.head(3)

,LOCNombre,PopLoc,UTAM,LOCid,ESTRATOPre,HOGARES,UTAMNombre,UTAMArea,PopDen[p/km2],geometry,originated Trips,received Trips,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer
0,usaquen,571268.0,1,1,6.0,913.0,PASEO DE LOS LIBERTADORES,6.301165e+06,540.118849,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",163,166,0,0,0,0,0,2,416
1,suba,1252675.0,2,11,6.0,891.0,LA ACADEMIA,6.711816e+06,527.173778,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",287,286,0,1,13,0,0,12,73
2,suba,1252675.0,3,11,6.0,452.0,GUAYMARAL,4.530358e+06,396.206927,"POLYGON ((-74.03501 4.82553, -74.03501 4.82552...",148,151,0,0,0,0,0,0,3


### Estimated Rt for every locality 

In [28]:
# Reading localities R_t
base_path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/RT_outputs/'

loc_R_list = ['usaquen.pkl', 'chapinero.pkl', 'santafe.pkl', 'sancristobal.pkl', 'usme.pkl',
             'tunjuelito.pkl', 'bosa.pkl', 'kennedy.pkl', 'fontibon.pkl', 'engativa.pkl',
             'suba.pkl', 'barriosunidos.pkl', 'teusaquillo.pkl', 'losmartires.pkl', 'antonionariño.pkl',
             'puentearanda.pkl', 'lacandelaria.pkl', 'rafaeluribeuribe.pkl', 'ciudadbolivar.pkl']

# Create DataFrame that contains all of estimated RT for each locality through time
R_list = []

for i in range(len(loc_R_list)):
    path_file = base_path+loc_R_list[i]
    R_list.append(pd.read_pickle(path_file))
    
for i in range(len(loc_R_list)):
    R_list[i].reset_index(drop=False, inplace=True)
    
R_df = pd.DataFrame(index = R_list[0]['Time Stamp'])
R_df.reset_index(drop=False, inplace=True)

for i in range(len(loc_R_list)):
    R_df[loc_R_list[i]] = 0
    R_df[loc_R_list[i]] = R_list[i]['R'].tolist()
    
#R_df.fillna(0)
R_df.head(5)

,Time Stamp,usaquen.pkl,chapinero.pkl,santafe.pkl,sancristobal.pkl,usme.pkl,tunjuelito.pkl,bosa.pkl,kennedy.pkl,fontibon.pkl,engativa.pkl,suba.pkl,barriosunidos.pkl,teusaquillo.pkl,losmartires.pkl,antonionariño.pkl,puentearanda.pkl,lacandelaria.pkl,rafaeluribeuribe.pkl,ciudadbolivar.pkl
0,2020-02-26,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2.634860,0.0,0.000000
1,2020-02-27,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,24.513241,24.513231,0.000000,0.0,0.0,0.0,0.0,1.825152,0.0,0.000000
2,2020-02-28,25.168460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,12.889200,12.864529,0.000000,0.0,0.0,0.0,0.0,1.825152,0.0,0.000000
3,2020-02-29,13.411527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.617419,9.256936,9.060387,25.617419,0.0,0.0,0.0,0.0,0.280187,0.0,25.617427
4,2020-03-01,10.052326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.604958,7.168667,6.864376,13.604958,0.0,0.0,0.0,0.0,0.000000,0.0,13.604997


In [29]:
# Save Base clustering DataFrame 
spatial_utam.to_pickle('/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/Outputs/partial_spatial_utam.pkl', compression='gzip')

### Active Cases Count

In [30]:
fechas_df = pd.read_csv('/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/Outputs/fechas_sintomas.csv')
fechas_df = fechas_df.rename(columns={'LOCALIDAD_ASIS': 'CASOS_REPORTADOS'})#Rename Column

#Change str to datetime objects
fechas_df['FECHA_DE_INICIO_DE_SINTOMAS'] = pd.to_datetime(fechas_df['FECHA_DE_INICIO_DE_SINTOMAS'], format='%Y-%m-%d')
fechas_df = fechas_df.sort_values(by='FECHA_DE_INICIO_DE_SINTOMAS', ascending=True)
fechas_df = fechas_df.reset_index(drop=True)

#Drop 0 and 21 id codes since they don´t belong to Bogotá geography 
index_0 = fechas_df[ fechas_df['CODIGO_LOCALIDAD'] == 0 ].index
index_21 = fechas_df[ fechas_df['CODIGO_LOCALIDAD'] == 21 ].index

fechas_df.drop(index_0, inplace = True)#drop fuera de bogotá and sin dato rows since we cannot calculate infection 
fechas_df.drop(index_21, inplace = True)# density for them (we don´t know their reference population)

fechas_df = fechas_df.reset_index(drop = True)

In [31]:
com_index = len(fechas_df['FECHA_DE_INICIO_DE_SINTOMAS']) #column length 
start_date = fechas_df.loc[0, 'FECHA_DE_INICIO_DE_SINTOMAS'] #start of the pandemic in Bogotá 
end_date = fechas_df.loc[com_index-1, 'FECHA_DE_INICIO_DE_SINTOMAS'] #Last reported date 

pan_days = end_date - start_date
pan_days = int(pan_days.days) #Days passed since pandemic start to last reported date grid x axis 
local_bog = 20 # number of rows y axis we are not considering 0 and 21 codes 

#create grid and fill it with reported cases y axis correspond to Bogota localities id code, x axis correspond to 
#number of days passed since pandemic started that way 0 index -> 2020-02-06, 1->2020-02-07 and so on

grid = np.ndarray([local_bog, pan_days+1]) #create grid
grid.fill(0) #fill grid with 0 

#Method used to fill grid with reported cases 
def fillGrid(date, code, cases):
    col = date - start_date
    col = int(col.days)
    grid[code-1][col] = cases
    

#Fill grid with cases 
fechas_df.apply(lambda row: fillGrid(row['FECHA_DE_INICIO_DE_SINTOMAS'], row['CODIGO_LOCALIDAD'], int(row['CASOS_REPORTADOS']) ), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
11293    None
11294    None
11295    None
11296    None
11297    None
Length: 11298, dtype: object

In [34]:
# Function used to count active cases
def ActiveCases (date, code):
    col = date - start_date
    col = int(col.days)
    if col >= 15:
        Active = sum(grid[code-1][col-15:col+1])
    else: 
        Active = sum(grid[code-1][0:col+1])
    return int(Active)

In [35]:
# Peak dates of infection in Bogotá to make some tests with them before applying ML to all data.
peak_dates = ['01/05/2020' ,'01/08/2020', '01/11/2020', '10/01/2021', '01/03/2021', '01/05/2021',
             '10/06/2021', '01/09/2021']

peak_dates2 = ['01-05-2020' ,'01-08-2020', '01-11-2020', '10-01-2021', '01-03-2021', '01-05-2021',
             '10-06-2021', '01-09-2021']

### Create DF for specific dates in order to perform clustering

In [36]:
# Select date
k = 3
selec_date = dt.datetime.strptime(peak_dates[k], "%d/%m/%Y")

#Add Active cases in interest date
Active_cases = []
for i in range(len(Loc_code)):
    Active_cases.append(ActiveCases(selec_date, Loc_code[i]))
    
#Add Active Cases
spatial_utam_date = spatial_utam.copy()

spatial_utam_date['cases'+peak_dates2[k]] = spatial_utam_date.apply(lambda row: Active_cases[row['LOCid']-1], axis=1)

In [37]:
# Granulate cases from locality to UTAM level
def single_cases(weight, caseLoc, PobLoc, ID, Hog):
    x = (100*caseLoc)/(PobLoc)
    y = (x*weight[ID-1]*Hog)/100
    return y

spatial_utam_date['UTAM_cases'+peak_dates2[k]] = spatial_utam_date.apply(lambda row: single_cases(ppl_weight, 
                                                                                                  row['cases'+peak_dates2[k]], 
                                                                                                  row['PopLoc'], row['LOCid'], 
                                                                                                  row['HOGARES']), axis=1)

In [38]:
# Compare mean granulated cases vs Locality cases
mean_cases = []

def ECM(R_Bog, R_Al):
    ecm = 0
    for i in range(len(R_Bog)):
        ecm += (R_Al[i] - R_Bog[i])**2
    ecm = ecm/len(R_Bog)
    return ecm 

for i in range(len(Loc_names)):
    temp = spatial_utam_date[ spatial_utam_date['LOCNombre'] == Loc_names[i] ]
    temp_list = temp['UTAM_cases'+peak_dates2[k]].values.tolist()
    cases_mean = sum(temp_list)
    mean_cases.append(cases_mean)    

In [39]:
np.array(mean_cases)

array([ 6877.,  2328.,  1410.,  3105.,  2641.,  1854.,  4908., 11022.,
        5178.,  9814., 13664.,  1776.,  2302.,  1106.,  1174.,  3146.,
         463.,  3530.,  4373.])

In [40]:
# They are the same so proportionality is being used correctly
np.array(Active_cases)

array([ 6877,  2328,  1410,  3105,  2641,  1854,  4908, 11022,  5178,
        9814, 13664,  1776,  2302,  1106,  1174,  3146,   463,  3530,
        4373])

In [41]:
#select Rt on specific date
rt_list = R_df[ R_df['Time Stamp'] == selec_date ]
rt_list = rt_list.iloc[:, 1:].values.tolist()[0]

#Add Rt on specific date
spatial_utam_date['Rt'+peak_dates2[k]] = spatial_utam_date.apply(lambda row: rt_list[row['LOCid']-1], axis=1)

spatial_utam_date.head(2)

,LOCNombre,PopLoc,UTAM,LOCid,ESTRATOPre,HOGARES,UTAMNombre,UTAMArea,PopDen[p/km2],geometry,...,N_Hosp,N_IPS,N_Col,N_PlazMer,N_ITur,N_SITP,N_Ecomer,cases10-01-2021,UTAM_cases10-01-2021,Rt10-01-2021
0,usaquen,571268.0,1,1,6.0,913.0,PASEO DE LOS LIBERTADORES,6.301165e+06,540.118849,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",...,0,0,0,0,0,2,416,6877,40.970316,1.018749
1,suba,1252675.0,2,11,6.0,891.0,LA ACADEMIA,6.711816e+06,527.173778,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",...,0,1,13,0,0,12,73,13664,38.595199,1.529611


In [42]:
#Generate final DF to apply clustering tests
spatial_utam_date['Total trips'] = spatial_utam_date['originated Trips']+spatial_utam_date['received Trips']

spatial_utam_date.drop(columns={'HOGARES', 'UTAMArea', 'originated Trips', 'received Trips',
                               'N_PlazMer', 'N_Col'}, inplace=True)

spatial_utam_date.head(2)

,LOCNombre,PopLoc,UTAM,LOCid,ESTRATOPre,UTAMNombre,PopDen[p/km2],geometry,N_Hosp,N_IPS,N_ITur,N_SITP,N_Ecomer,cases10-01-2021,UTAM_cases10-01-2021,Rt10-01-2021,Total trips
0,usaquen,571268.0,1,1,6.0,PASEO DE LOS LIBERTADORES,540.118849,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0,0,0,2,416,6877,40.970316,1.018749,329
1,suba,1252675.0,2,11,6.0,LA ACADEMIA,527.173778,"POLYGON ((-74.03849 4.79952, -74.04214 4.77702...",0,1,0,12,73,13664,38.595199,1.529611,573


In [43]:
# Normalize values 
spatial_utam_date_rt = spatial_utam_date.copy()

spatial_utam_date_rt['ESTRATOPre'] = spatial_utam_date_rt['ESTRATOPre']/(spatial_utam_date_rt['ESTRATOPre'].max()*4)
spatial_utam_date_rt['PopDen[p/km2]'] = spatial_utam_date_rt['PopDen[p/km2]']/(spatial_utam_date_rt['PopDen[p/km2]'].max()*4)
spatial_utam_date_rt['N_Hosp'] = spatial_utam_date_rt['N_Hosp']/(spatial_utam_date_rt['N_Hosp'].max()*4)
spatial_utam_date_rt['N_IPS'] = spatial_utam_date_rt['N_IPS']/(spatial_utam_date_rt['N_IPS'].max()*4)
spatial_utam_date_rt['N_ITur']  = spatial_utam_date_rt['N_ITur']/(spatial_utam_date_rt['N_ITur'].max()*4)
spatial_utam_date_rt['N_SITP'] = spatial_utam_date_rt['N_SITP']/(spatial_utam_date_rt['N_SITP'].max()*4)
spatial_utam_date_rt['N_Ecomer'] = spatial_utam_date_rt['N_Ecomer']/(spatial_utam_date_rt['N_Ecomer'].max()*4)
spatial_utam_date_rt['Total trips'] = spatial_utam_date_rt['Total trips']/(spatial_utam_date_rt['Total trips'].max()*4)
spatial_utam_date_rt['UTAM_cases'+peak_dates2[k]] = spatial_utam_date_rt['UTAM_cases'+peak_dates2[k]]/(spatial_utam_date_rt['UTAM_cases'+peak_dates2[k]].max()*3)

In [44]:
# Function to granulate RT in UTAMS
def singularizeRT(loc, estrato, popden, hosp, ips, itur, sitp, comer, trips, cases, RT):
    if loc == 'candelaria':
        return RT
    else:
        a_rt = RT-estrato+popden-hosp-ips+itur+sitp+comer+trips+cases
        return a_rt


In [45]:
# Granulate RT
spatial_utam_date_rt['singleRT_'+peak_dates2[k]] = spatial_utam_date_rt.apply(lambda row: singularizeRT(row['LOCNombre'], row['ESTRATOPre'], 
                                                                                       row['PopDen[p/km2]'], row['N_Hosp'],
                                                                                       row['N_IPS'], row['N_ITur'],
                                                                                       row['N_SITP'], row['N_Ecomer'],
                                                                                       row['Total trips'], 
                                                                                       row['UTAM_cases'+peak_dates2[k]],
                                                                                       row['Rt'+peak_dates2[k]]), axis=1)

In [46]:
# Localities name list
name_list = ['usaquen', 'chapinero', 'santa fe', 'san cristobal', 'usme', 'tunjuelito', 'bosa', 
             'kennedy', 'fontibon', 'engativa', 'suba', 'barrios unidos', 'teusaquillo', 
             'los martires', 'antonio narino', 'puente aranda', 'candelaria', 'rafael uribe uribe', 'ciudad bolivar']

In [47]:
# Compare mean Adjusted Rt vs Locality Rt
mean_rt = []
# Mean square error function
def ECM(R_Bog, R_Al):
    ecm = 0
    for i in range(len(R_Bog)):
        ecm += (R_Al[i] - R_Bog[i])**2
    ecm = ecm/len(R_Bog)
    return ecm 
# mean rt from granulated RT in UTAMs inside localities
for i in range(len(name_list)):
    temp = spatial_utam_date_rt[ spatial_utam_date_rt['LOCNombre'] == name_list[i] ]
    temp_list = temp['singleRT_'+peak_dates2[k]].values.tolist()
    rt_mean = sum(temp_list)/(len(temp_list))
    mean_rt.append(rt_mean)

In [48]:
# Compare them visually 
np.array(rt_list)

array([1.01874938, 1.12154511, 0.79741259, 0.89510453, 0.6722215 ,
       0.96218664, 1.35199551, 0.95548613, 1.01107834, 1.176654  ,
       1.52961144, 1.33335708, 0.77800302, 0.63826137, 0.71182792,
       0.94563128, 0.90831375, 1.34331024, 1.22999576])

In [49]:
np.array(mean_rt)

array([1.10126274, 1.25709879, 0.9987852 , 1.11569446, 0.79308976,
       1.16482101, 1.65425225, 1.20768683, 1.09307528, 1.4144078 ,
       1.69212832, 1.44794442, 0.84461382, 0.96459559, 0.89485601,
       1.11433212, 0.90831375, 1.58592452, 1.3900965 ])

In [50]:
# MSE between localities
ECM(mean_rt, rt_list)

0.036063412058567515

In [51]:
# Finally generate final normalized DF with an estimated rt for each utam 
spatial_utam_date['local_rt_'+peak_dates2[k]] = spatial_utam_date_rt['singleRT_'+peak_dates2[k]]

spatial_utam_date['ESTRATOPre'] = spatial_utam_date['ESTRATOPre']/(spatial_utam_date['ESTRATOPre'].max())
spatial_utam_date['PopDen[p/km2]'] = spatial_utam_date['PopDen[p/km2]']/(spatial_utam_date['PopDen[p/km2]'].max())
spatial_utam_date['N_Hosp'] = spatial_utam_date['N_Hosp']/(spatial_utam_date['N_Hosp'].max())
spatial_utam_date['N_IPS'] = spatial_utam_date['N_IPS']/(spatial_utam_date['N_IPS'].max())
spatial_utam_date['N_ITur']  = spatial_utam_date['N_ITur']/(spatial_utam_date['N_ITur'].max())
spatial_utam_date['N_SITP'] = spatial_utam_date['N_SITP']/(spatial_utam_date['N_SITP'].max())
spatial_utam_date['N_Ecomer'] = spatial_utam_date['N_Ecomer']/(spatial_utam_date['N_Ecomer'].max())
spatial_utam_date['Total trips'] = spatial_utam_date['Total trips']/(spatial_utam_date['Total trips'].max())
spatial_utam_date['UTAM_cases'+peak_dates2[k]] = spatial_utam_date['UTAM_cases'+peak_dates2[k]]/(spatial_utam_date['UTAM_cases'+peak_dates2[k]].max())
spatial_utam_date['local_rt_'+peak_dates2[k]] = spatial_utam_date['local_rt_'+peak_dates2[k]]/(spatial_utam_date['local_rt_'+peak_dates2[k]].max())

In [52]:
# Drop specific cols
spatial_utam_date.drop(columns={'Rt'+peak_dates2[k], 'cases'+peak_dates2[k]}, inplace=True)
spatial_utam_date.head(1)

,LOCNombre,PopLoc,UTAM,LOCid,ESTRATOPre,UTAMNombre,PopDen[p/km2],geometry,N_Hosp,N_IPS,N_ITur,N_SITP,N_Ecomer,UTAM_cases10-01-2021,Total trips,local_rt_10-01-2021
0,usaquen,571268.0,1,1,1.0,PASEO DE LOS LIBERTADORES,0.010476,"POLYGON ((-74.02609 4.82311, -74.02571 4.82311...",0.0,0.0,0.0,0.009524,0.019154,0.011113,0.045404,0.337492


In [53]:
# Organize cols
spatial_utam_date = spatial_utam_date[ ['LOCNombre', 'PopLoc', 'UTAM', 'LOCid', 'ESTRATOPre', 'UTAMNombre',
                                       'PopDen[p/km2]', 'geometry', 'N_Hosp', 'N_IPS', 'N_ITur', 'N_SITP',
                                       'N_Ecomer', 'UTAM_cases'+peak_dates2[k], 'local_rt_'+peak_dates2[k] ,'Total trips'] ]

In [84]:
# several test DFs are needed if user wants to execute silhouette test
df_file_name = 'date_'+peak_dates2[k]+'.pkl'
spatial_utam_date.to_pickle('/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/Test_Clustering_DF/'+df_file_name, compression='gzip')